# COVID-19 cases in Kenya, preprocessing analysis

In [1]:
#load some key libraries

import pandas as pd
import datetime as dt
from datetime import date
import calendar
import plotly.express as px
import plotly.graph_objects as go
import matplotlib.pyplot as plt

In [2]:
#select_important_columns
#covid_data_original = pd.read_excel("Covid-19_cleaned_07-02-2022.xls", engine="xlrd")
covid_data_original = pd.read_table("Covid-19_cleaned_07-02-2022.txt")

covid_data_original["county_where_the_case_was_diagonised"] =  covid_data_original["county_where_the_case_was_diagonised"].\
                                                            apply(lambda x : str(x).title())
covid_data_original["county_where_the_case_was_diagonised"] = covid_data_original["county_where_the_case_was_diagonised"].\
                    replace({"Murang'A": "Murang'a", "Kakamega ": "Kakamega", "Nakuru ": "Nakuru", "Kajiado ":"Kajiado"})


covid_data_original.head()

/var/folders/0x/t7x8m9311b1gnnb39j773h8c3q8zn2/T/ipykernel_16630/2193926576.py:3: DtypeWarning: Columns (2,3,15,16,17,25,26,28,29,35,36,37,38,39,40,42,45,46,47,48,49,50,51,52) have mixed types. Specify dtype option on import or set low_memory=False.
  covid_data_original = pd.read_table("Covid-19_cleaned_07-02-2022.txt")


,unique_id,case_id,contact_of_new,age_years,sex,occupation_sector_job_title_employer_,county_of_residence,county_where_the_case_was_diagonised,nationality,sub_county,...,co_morbidity,management,status_need,date_of_announcement_by_cs,type_of_testing,graph_date,status,total_deaths,history_of_contact_new,symptomatic_yes_no_new
0,Case-01,Case 1,NaN,27.0,F,/Unemployed,Kajiado,Kajiado,Kenyan,Kajiado North,...,NaN,NaN,NaN,13-Mar-20,PCR,NaN,#ERROR!,1241,NaN,YES
1,Case-02,Case 2,NaN,48.0,F,/Unknown,Nairobi,Nairobi,Kenyan,Starehe,...,NaN,NaN,NaN,NaN,PCR,NaN,#ERROR!,NaN,NaN,YES
2,Case-03,Case 3,Case-01,27.0,M,/Unknown,Nairobi,Nairobi,Kenyan,Kajiado North,...,NaN,NaN,NaN,NaN,PCR,NaN,#ERROR!,NaN,YES,YES
3,Case-04,Case 4,NaN,31.0,F,/Unknown,Nairobi,Nairobi,Kenyan,Langata,...,NaN,NaN,NaN,NaN,PCR,NaN,#ERROR!,NaN,NaN,YES
4,Case-05,Case 5,NaN,42.0,F,/Unknown,Nairobi,Nairobi,Kenyan,Westlands,...,NaN,NaN,NaN,NaN,PCR,NaN,#ERROR!,NaN,NaN,YES


Visualizing lab_results column returns presence of wrong labels and names of hospitals. Creating a new colum fpor confirmed positives. This will determine the total cases

In [3]:
covid_data = covid_data_original[["unique_id","age_years", "sex","county_of_residence","county_where_the_case_was_diagonised",
                                    "status_need","status","lab_results","date_of_lab_confirmation","outcome_death_Discharge_still_in_hospital_"]]

covid_data.tail()

,unique_id,age_years,sex,county_of_residence,county_where_the_case_was_diagonised,status_need,status,lab_results,date_of_lab_confirmation,outcome_death_Discharge_still_in_hospital_
322122,Case-0322092,21.0,F,Siaya,Siaya,NaN,NaN,NaN,05/02/2022,NaN
322123,Case-0322093,2.0,M,Siaya,Siaya,NaN,NaN,NaN,05/02/2022,NaN
322124,Case-0322094,70.0,M,Tharaka Nithi,Tharaka Nithi,NaN,NaN,NaN,05/02/2022,NaN
322125,Case-0322095,15.0,F,Uasin Gishu,Uasin Gishu,NaN,NaN,NaN,05/02/2022,NaN
322126,Case-0322096,43.0,M,Uasin Gishu,Uasin Gishu,NaN,NaN,NaN,05/02/2022,NaN


In [4]:
#convert the date column to true dates
covid_data["date_of_lab_confirmation"] = pd.to_datetime(covid_data["date_of_lab_confirmation"], format="%d/%m/%Y", errors="coerce")

/var/folders/0x/t7x8m9311b1gnnb39j773h8c3q8zn2/T/ipykernel_16630/2772849741.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  covid_data["date_of_lab_confirmation"] = pd.to_datetime(covid_data["date_of_lab_confirmation"], format="%d/%m/%Y", errors="coerce")


## Plots based on lab_results and outcome from hospitals

In [5]:
covid_data_view = covid_data[[ "sex","age_years", "lab_results", "date_of_lab_confirmation", "outcome_death_Discharge_still_in_hospital_" , "county_where_the_case_was_diagonised"]]

#there exists some cases with missing date of lab confirmation. Dropping them
covid_data_view=covid_data_view.dropna(subset=["date_of_lab_confirmation"])

covid_data_view.tail()

,sex,age_years,lab_results,date_of_lab_confirmation,outcome_death_Discharge_still_in_hospital_,county_where_the_case_was_diagonised
322122,F,21.0,NaN,2022-02-05,NaN,Siaya
322123,M,2.0,NaN,2022-02-05,NaN,Siaya
322124,M,70.0,NaN,2022-02-05,NaN,Tharaka Nithi
322125,F,15.0,NaN,2022-02-05,NaN,Uasin Gishu
322126,M,43.0,NaN,2022-02-05,NaN,Uasin Gishu


In [6]:
covid_data_view["Year"] = covid_data_view["date_of_lab_confirmation"].dt.to_period("Y")
covid_data_view["Month"] =  covid_data_view["date_of_lab_confirmation"].dt.month
covid_data_view["Week"] = covid_data_view["date_of_lab_confirmation"].dt.week
covid_data_view["Month"] =  covid_data_view["Month"].apply(lambda x: calendar.month_abbr[x])
covid_data_view["Month_year"] = covid_data_view["date_of_lab_confirmation"].dt.to_period("M")
covid_data_view["year_month"] = covid_data_view["Month"] + "-" +  covid_data_view["Year"].astype(str) 
covid_data_view.head()

/var/folders/0x/t7x8m9311b1gnnb39j773h8c3q8zn2/T/ipykernel_16630/3999872376.py:3: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated. Please use Series.dt.isocalendar().week instead.
  covid_data_view["Week"] = covid_data_view["date_of_lab_confirmation"].dt.week


,sex,age_years,lab_results,date_of_lab_confirmation,outcome_death_Discharge_still_in_hospital_,county_where_the_case_was_diagonised,Year,Month,Week,Month_year,year_month
0,F,27.0,Positive,2020-03-12,Discharge,Kajiado,2020,Mar,11,2020-03,Mar-2020
1,F,48.0,Positive,2020-03-14,Discharge,Nairobi,2020,Mar,11,2020-03,Mar-2020
2,M,27.0,Positive,2020-03-14,Discharge,Nairobi,2020,Mar,11,2020-03,Mar-2020
3,F,31.0,Positive,2020-03-17,Discharge,Nairobi,2020,Mar,12,2020-03,Mar-2020
4,F,42.0,Positive,2020-03-17,Discharge,Nairobi,2020,Mar,12,2020-03,Mar-2020


In [7]:
Monthly_cases = covid_data_view[["Month_year", "year_month", "outcome_death_Discharge_still_in_hospital_" ,"lab_results"]].\
    groupby(["Month_year", "year_month"])["Month_year"].count().to_frame(name = "COVID19_Cases").reset_index()
Monthly_cases.rename(columns = {"COVID19_Cases": "Reported_cases"}, inplace=True)

death_cases = covid_data_view[covid_data_view["outcome_death_Discharge_still_in_hospital_"] == "Dead"]
monthly_death_cases = death_cases[["outcome_death_Discharge_still_in_hospital_","Month_year", "year_month"]].\
    groupby(["Month_year", "year_month"]).count().reset_index()
monthly_death_cases.rename(columns = {"outcome_death_Discharge_still_in_hospital_" : "Death_cases"}, inplace=True)

covid_monthly_data = pd.merge(Monthly_cases, monthly_death_cases, on = "year_month", how = "outer")
covid_monthly_data =  covid_monthly_data.drop("Month_year_x", axis=1)
covid_monthly_data =  covid_monthly_data.rename(columns={"Month_year_y" : "Month_year"})
covid_monthly_data["Month_year"]= covid_monthly_data["Month_year"].dt.strftime("%Y-%m")

covid_monthly_data.to_csv("covid_monthly_data.csv")


In [8]:
Monthly_cases = pd.read_csv("covid_monthly_data.csv")

Monthly_cases.head()

,Unnamed: 0,year_month,Reported_cases,Month_year,Death_cases
0,0,Mar-2020,84,2020-03,5
1,1,Apr-2020,350,2020-04,20
2,2,May-2020,1701,2020-05,66
3,3,Jun-2020,4719,2020-06,134
4,4,Jul-2020,14480,2020-07,347


Daily Cases

In [9]:
#summary for daily reported dcases
daily_cases =  covid_data_view[["date_of_lab_confirmation", "lab_results"]].groupby("date_of_lab_confirmation")["date_of_lab_confirmation"].\
    count().to_frame(name = "Reported_Cases").reset_index()
daily_cases.rename(columns = {"date_of_lab_confirmation": "Date"}, inplace=True)
#daily_cases.tail()

#summary for dail deaths
daily_deaths = covid_data[covid_data["outcome_death_Discharge_still_in_hospital_"] == "Dead"]
daily_deaths = daily_deaths[[ "date_of_lab_confirmation", "outcome_death_Discharge_still_in_hospital_" ]].\
    groupby("date_of_lab_confirmation")["date_of_lab_confirmation"].count().to_frame(name="death_cases").reset_index()
daily_deaths.rename(columns = {"date_of_lab_confirmation": "Date"}, inplace=True)

#summary for dicharged
daily_discharges = covid_data[covid_data["outcome_death_Discharge_still_in_hospital_"] == "Discharge"]
daily_discharges = daily_discharges[[ "date_of_lab_confirmation", "outcome_death_Discharge_still_in_hospital_" ]].\
    groupby("date_of_lab_confirmation")["date_of_lab_confirmation"].count().to_frame(name="Reported_discharges").reset_index()
daily_discharges.rename(columns = {"date_of_lab_confirmation": "Date"}, inplace=True)

covid_daily_data = pd.merge(pd.merge(daily_cases, daily_deaths, how = "outer"), daily_discharges, how="outer")

covid_daily_data["death_cases"] = covid_daily_data["death_cases"].fillna(0)
covid_daily_data["Reported_discharges"] = covid_daily_data["Reported_discharges"].fillna(0)

covid_daily_data = covid_daily_data.astype({"Reported_Cases":int, "death_cases": int, "Reported_discharges":int })

covid_daily_data.to_csv("covid_daily_data.csv")
covid_daily_data.head()

,Date,Reported_Cases,death_cases,Reported_discharges
0,2020-03-12,1,0,1
1,2020-03-14,2,0,2
2,2020-03-17,4,0,4
3,2020-03-19,3,0,3
4,2020-03-20,1,0,1


In [10]:
from plotly.graph_objects import Layout
layout = Layout(plot_bgcolor="rgba(0,0,0,0)")
fig = go.Figure(layout=layout)
fig.add_trace(go.Scatter( x = covid_daily_data["Date"],  y = covid_daily_data["Reported_Cases"],mode='none',
                            fill = 'tozeroy', marker = dict(color = "#A3A3A3")))
fig.update_layout(title = "COVID-19 Monthly Cases", font_color = "darkcyan")
fig.update_yaxes(title =  "Counts", showline=True, linewidth = 0.2, linecolor = "gray", gridcolor = "gainsboro")
fig.update_xaxes(title = "Period", showgrid=False)
fig.show()

Weekly Cases

In [11]:
#export data

covid_data.to_csv("covid_data_processed.csv")

In [12]:
#filter data for those who died
dead = covid_data[covid_data["outcome_death_Discharge_still_in_hospital_"] == "Dead"]
daily_deaths = dead[[ "date_of_lab_confirmation", "outcome_death_Discharge_still_in_hospital_" ]].\
    groupby("date_of_lab_confirmation")["date_of_lab_confirmation"].count().to_frame(name="death_cases").reset_index()
daily_deaths.rename(columns = {"date_of_lab_confirmation": "Date"}, inplace=True)

daily_deaths.to_csv("death_cases.csv")

from plotly.graph_objects import Layout
layout = Layout(plot_bgcolor="rgba(0,0,0,0)")
fig5 = go.Figure(layout=layout)
fig5.add_trace(go.Scatter( x = daily_deaths.Date,  y = daily_deaths.death_cases,mode='none',
                            fill = 'tozeroy', marker = dict(color = "#A3A3A3")))
fig5.update_layout(title = "COVID-19 Daily Deaths", font_color = "darkcyan")
fig5.update_yaxes(title =  "Deaths Cases", showline=True, linewidth = 0.2, linecolor = "gray", gridcolor = "gainsboro")
fig5.update_xaxes(title = "Period", showgrid=False)
#fig.show()

## Visualization Based on County where case was diagnoised


In [13]:
covid_data_view.head()


,sex,age_years,lab_results,date_of_lab_confirmation,outcome_death_Discharge_still_in_hospital_,county_where_the_case_was_diagonised,Year,Month,Week,Month_year,year_month
0,F,27.0,Positive,2020-03-12,Discharge,Kajiado,2020,Mar,11,2020-03,Mar-2020
1,F,48.0,Positive,2020-03-14,Discharge,Nairobi,2020,Mar,11,2020-03,Mar-2020
2,M,27.0,Positive,2020-03-14,Discharge,Nairobi,2020,Mar,11,2020-03,Mar-2020
3,F,31.0,Positive,2020-03-17,Discharge,Nairobi,2020,Mar,12,2020-03,Mar-2020
4,F,42.0,Positive,2020-03-17,Discharge,Nairobi,2020,Mar,12,2020-03,Mar-2020


In [14]:
covid_data.head()

,unique_id,age_years,sex,county_of_residence,county_where_the_case_was_diagonised,status_need,status,lab_results,date_of_lab_confirmation,outcome_death_Discharge_still_in_hospital_
0,Case-01,27.0,F,Kajiado,Kajiado,NaN,#ERROR!,Positive,2020-03-12,Discharge
1,Case-02,48.0,F,Nairobi,Nairobi,NaN,#ERROR!,Positive,2020-03-14,Discharge
2,Case-03,27.0,M,Nairobi,Nairobi,NaN,#ERROR!,Positive,2020-03-14,Discharge
3,Case-04,31.0,F,Nairobi,Nairobi,NaN,#ERROR!,Positive,2020-03-17,Discharge
4,Case-05,42.0,F,Nairobi,Nairobi,NaN,#ERROR!,Positive,2020-03-17,Discharge


In [15]:
covid_data["county_where_the_case_was_diagonised"].unique()

array(['Kajiado', 'Nairobi', 'Kwale', 'Kilifi', 'Mombasa', 'Kakamega',
       'Kitui', 'Nakuru', 'Mandera', 'Uasin Gishu', 'Siaya', 'Homa Bay',
       'Kiambu', 'Wajir', 'Makueni', 'Migori', 'Bungoma', 'Busia',
       'Isiolo', 'Machakos', 'Bomet', 'Taita Taveta', 'Garissa', 'Kisii',
       'Meru', 'Kirinyaga', 'Trans Nzoia', 'Kisumu', 'Turkana', 'Embu',
       'Nyeri', "Murang'a", 'Kericho', 'Laikipia', 'Nandi',
       'Elgeyo Marakwet', 'Marsabit', 'Narok', 'Nyamira', 'Vihiga',
       'Lamu', 'Nyandarua', 'Tharaka Nithi', 'Tana River', 'Nan',
       'Sore Throat, General Body Malaise, Stomach Upset.', 'Baringo',
       'West Pokot', 'Samburu', '13/08/2020', '02/09/2020', '13/10/2020',
       '15/10/2020', '16/10/2020', '0021-01-27', 'Makueni ', '0021-03-06',
       '0021-03-08', '0021-03-13'], dtype=object)

In [16]:
#covid_data.rename(columns = {"county_where_the_case_was_diagonised" :"County"}, inplace=True)
county_death = covid_data[covid_data["outcome_death_Discharge_still_in_hospital_"] == "Dead"]
#county_death.rename(columns = {"county_where_the_case_was_diagonised" :"County"}, inplace=True)

county_death_counts = county_death[["county_where_the_case_was_diagonised", "outcome_death_Discharge_still_in_hospital_"]].\
                        groupby("county_where_the_case_was_diagonised").count().reset_index()
                        
county_death_counts.rename(columns={"outcome_death_Discharge_still_in_hospital_": "Death_cases"}, inplace=True)
#county_death_counts

discharged = covid_data[covid_data["outcome_death_Discharge_still_in_hospital_"] == "Discharge"]
discharged = discharged[["county_where_the_case_was_diagonised", "outcome_death_Discharge_still_in_hospital_"]].\
                    groupby("county_where_the_case_was_diagonised").count().reset_index()
discharged.rename(columns={"outcome_death_Discharge_still_in_hospital_": "Discharged"}, inplace=True)
#discharged

#covid_data["county_of_residence"] = covid_data["county_where_the_case_was_diagonised"].str.strip().str.title()#.unique() #Needs clean-up for naming consistency
county_prevalence = covid_data.groupby("county_where_the_case_was_diagonised")["county_where_the_case_was_diagonised"].count().\
                to_frame(name = "cases").reset_index()
county_prevalence = county_prevalence.sort_values("cases", ascending=False)
#county_prevalence

county_cases = pd.merge(pd.merge(county_prevalence, county_death_counts, on="county_where_the_case_was_diagonised"),\
                    discharged,how="left", on="county_where_the_case_was_diagonised")

county_cases["Discharged"] = county_cases["Discharged"].fillna(0)
county_cases = county_cases.astype({"cases":int, "Death_cases": int, "Discharged":int })

county_cases.rename(columns={"county_where_the_case_was_diagonised": "County"}, inplace=True)
county_cases.to_csv("cases_per_county.csv")

county_cases.head()


,County,cases,Death_cases,Discharged
0,Nairobi,130228,1542,3561
1,Kiambu,20127,545,263
2,Mombasa,17887,337,1530
3,Nakuru,16947,531,66
4,Uasin Gishu,10507,241,111


### Visualize as maps

In [17]:
import geopandas as gpd
#import geoplot as gplt
#import geoplot.crs as gcrs
#import imageio
import matplotlib.pyplot as plt
#import mpclassift as pc



In [18]:
#get kenya shaoe files

kenya_shape = gpd.read_file("kenyan-counties/County.shp")


In [19]:
# kenya_shape["COUNTY"] =  kenya_shape["COUNTY"].replace("Keiyo-Marakwet", "Elgeyo Marakwet")
# kenya_shape["COUNTY"] =  kenya_shape["COUNTY"].replace("Tharaka", "Tharaka Nithi")
# county_prevalence["County"] = county_prevalence["County"].replace( "Murang'A", "Murang'a" )
# #kenya_shape.head()

In [20]:
# county_shape = pd.merge(kenya_shape, county_prevalence, how="left", left_on=["COUNTY"], right_on=["County"])
# county_shape.head()

# county_shape.to_csv("county_cases.csv")
# county_shape.head()


# creating regions from county

In [21]:
county_cases.head()

,County,cases,Death_cases,Discharged
0,Nairobi,130228,1542,3561
1,Kiambu,20127,545,263
2,Mombasa,17887,337,1530
3,Nakuru,16947,531,66
4,Uasin Gishu,10507,241,111


In [22]:

regions = dict(
    {"Coast" : ["Mombasa", "Kwale", "Kilifi", "Tana River", "Taita Taveta", "Lamu"],
    "North Eastern" : [  "Garissa", "Wajir", "Mandera"],
    "Eastern" : ["Marsabit", "Isiolo", "Meru", "Tharaka Nithi", "Embu", "Kitui", "Machakos", "Makueni"],
    "Central" : [ "Nyandarua", "Murang'a", "Kirinyaga", "Nyeri", "Kiambu"],
    "Rift Valley" : ["Nakuru", "Narok", "Kajiado", "Kericho", "Bomet", "Turkana", "West Pokot", "Samburu","Trans Nzoia", "Uasin Gishu", "Elgeyo Marakwet", "Nandi", "Baringo", "Laikipia"],
    "Western": ["Kakamega", "Vihiga", "Bungoma", "Busia"],
    "Nyanza": [ "Siaya", "Kisumu", "Homa Bay", "Migori", "Kisii", "Nyamira"],
    "Nairobi": ["Nairobi"]
    }) 

#print(regions.values())
province = []
for name in county_cases["County"]:
    for (k,v) in regions.items():
        for i in v:
            if i == name:
                province.append(k)
        

county_cases["province"] =  province

county_cases.head()

,County,cases,Death_cases,Discharged,province
0,Nairobi,130228,1542,3561,Nairobi
1,Kiambu,20127,545,263,Central
2,Mombasa,17887,337,1530,Coast
3,Nakuru,16947,531,66,Rift Valley
4,Uasin Gishu,10507,241,111,Rift Valley


In [23]:
province_cases = county_cases.groupby(["province"])["cases","Death_cases", "Discharged"].sum().reset_index()

kenya_province = gpd.read_file("Admin2/KEN_admin2_2002_DEPHA.shp")
kenya_province["PROVINCE"] =  kenya_province["PROVINCE"].str.replace("\r\n", "").str.title()

province_cases = pd.merge(kenya_province, province_cases, how="left", left_on=["PROVINCE"], right_on=["province"])

province_cases.to_csv("provice_level_cases.csv")


province_cases

/var/folders/0x/t7x8m9311b1gnnb39j773h8c3q8zn2/T/ipykernel_16630/2087339027.py:1: FutureWarning:

Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.



,PROVINCE,Prov_Pcode,geometry,province,cases,Death_cases,Discharged
0,Central,KE2,"POLYGON ((37.31979 -0.75808, 37.31935 -0.75773...",Central,35648,1085,284
1,Coast,KE3,"MULTIPOLYGON (((39.06581 -3.02189, 39.05392 -2...",Coast,29419,439,1684
2,Eastern,KE4,"POLYGON ((38.95712 2.08819, 38.95787 2.08736, ...",Eastern,25266,634,193
3,Nairobi,KE1,"POLYGON ((36.68167 -1.29539, 36.68264 -1.29358...",Nairobi,130228,1542,3561
4,North Eastern,KE5,"POLYGON ((40.97417 2.15810, 40.97480 2.15811, ...",North Eastern,3381,138,67
5,Nyanza,KE6,"MULTIPOLYGON (((35.04765 -0.70610, 35.05234 -0...",Nyanza,25418,488,182
6,Rift Valley,KE7,"POLYGON ((36.60691 2.39510, 36.60953 2.39583, ...",Rift Valley,57786,1089,545
7,Western,KE8,"MULTIPOLYGON (((35.15522 0.68490, 35.15526 0.6...",Western,14947,206,442


In [24]:
province_cases_sort = province_cases[["province", "cases"]]
province_cases_sort = province_cases_sort.sort_values("cases", ascending = True)
fig = px.histogram(province_cases_sort,
               y="province",x="cases",
               orientation="h",title="Region Level Cases",
               height=500, width=900
               )
fig.update_layout(uniformtext_minsize = 3, font_color = "darkcyan", legend = dict(bgcolor = "white"), bargap =0.05)
fig.update_yaxes(title =  "Counts", showline=True, linewidth = 0.2, linecolor = "gray", gridcolor = "gainsboro")
fig.update_xaxes(title = "Period", showgrid=False)

#fig.show()

#fig.write_image("county_cases_image.png")

## Visualizing death cases

Prepare dataset to be used for county level comparison - time series

In [53]:
county_daily_data = covid_data_view[covid_data_view["lab_results"] == "Positive"]

county_daily_data = county_daily_data.rename(columns = { "outcome_death_Discharge_still_in_hospital_": "outcome", 
                            "county_where_the_case_was_diagonised" : "county"})

county_daily_data["sex"] = county_daily_data["sex"].replace({"m":"M", "f":"F"})

county_daily_data = county_daily_data[(county_daily_data["age_years"].notna()) & (county_daily_data["age_years"] != "M")]
county_daily_data["age_years"] = county_daily_data["age_years"].astype(int)
    
county_daily_data = county_daily_data[(county_daily_data["age_years"] > 0) & (county_daily_data["age_years"] < 100)]


county_daily_data.to_csv("county_daily_data.csv")
county_daily_data.head()

,sex,age_years,lab_results,date_of_lab_confirmation,outcome,county,Year,Month,Week,Month_year,year_month
0,F,27,Positive,2020-03-12,Discharge,Kajiado,2020,Mar,11,2020-03,Mar-2020
1,F,48,Positive,2020-03-14,Discharge,Nairobi,2020,Mar,11,2020-03,Mar-2020
2,M,27,Positive,2020-03-14,Discharge,Nairobi,2020,Mar,11,2020-03,Mar-2020
3,F,31,Positive,2020-03-17,Discharge,Nairobi,2020,Mar,12,2020-03,Mar-2020
4,F,42,Positive,2020-03-17,Discharge,Nairobi,2020,Mar,12,2020-03,Mar-2020


## Age based analysis with gender

In [35]:
age_gender = county_daily_data.reset_index()[["sex", "age_years", "lab_results"]]
age_gender =  age_gender[(age_gender["age_years"].notna()) & (age_gender["age_years"] != "M")]  #take rows that are not na
age_gender["age_years"] =  age_gender["age_years"].astype(int)
age_gender = age_gender[(age_gender["age_years"] > 0) & age_gender["age_years"] < 100]
bins  = [0,5,10,15,20,25,30,35,40,45,50,55,60,65,70,75,80,85,90,100 ]
labels = ["0-5", "6-10", "11-15","16-20","21-25", "25-30", "31-35", "36-40", "41-45","46-50","51-55","56-60", "61-65","66-70","71-75",
            "76-80","81-85","86-90","90+"]
age_gender["age_groups"] = pd.cut(age_gender.age_years, bins, labels=labels, include_lowest=True)

age_gender["sex"] = age_gender["sex"].replace({"m":"M", "f":"F"})
age_gender = age_gender.rename(columns={"sex":"Gender"})
age_gender = age_gender[["Gender","age_years","age_groups"]].groupby(["age_groups","Gender"]).count().reset_index()
age_gender = pd.pivot_table(age_gender, values="age_years", index = ["age_groups"],columns = ["Gender"])
age_gender.to_csv("age_gender_cases_data.csv")

In [30]:
county_daily_data_selected = county_daily_data[county_daily_data["county"].isin(["Nairobi", "Mombasa"])]
county_daily_data_selected = county_daily_data_selected[[ "county", "lab_results", "date_of_lab_confirmation"]].\
                                groupby(["county", "date_of_lab_confirmation"]).count().reset_index()

fig = px.line(county_daily_data_selected, x="date_of_lab_confirmation" , y = "lab_results" , color = "county")
fig.update_traces(mode = 'lines')
fig.update_layout()
fig.show()

In [31]:
county_daily_data.head()

,sex,age_years,lab_results,date_of_lab_confirmation,outcome,county,Year,Month,Week,Month_year,year_month
0,F,27.0,Positive,2020-03-12,Discharge,Kajiado,2020,Mar,11,2020-03,Mar-2020
1,F,48.0,Positive,2020-03-14,Discharge,Nairobi,2020,Mar,11,2020-03,Mar-2020
2,M,27.0,Positive,2020-03-14,Discharge,Nairobi,2020,Mar,11,2020-03,Mar-2020
3,F,31.0,Positive,2020-03-17,Discharge,Nairobi,2020,Mar,12,2020-03,Mar-2020
4,F,42.0,Positive,2020-03-17,Discharge,Nairobi,2020,Mar,12,2020-03,Mar-2020


In [32]:
county_daily_data.head()

,sex,age_years,lab_results,date_of_lab_confirmation,outcome,county,Year,Month,Week,Month_year,year_month
0,F,27.0,Positive,2020-03-12,Discharge,Kajiado,2020,Mar,11,2020-03,Mar-2020
1,F,48.0,Positive,2020-03-14,Discharge,Nairobi,2020,Mar,11,2020-03,Mar-2020
2,M,27.0,Positive,2020-03-14,Discharge,Nairobi,2020,Mar,11,2020-03,Mar-2020
3,F,31.0,Positive,2020-03-17,Discharge,Nairobi,2020,Mar,12,2020-03,Mar-2020
4,F,42.0,Positive,2020-03-17,Discharge,Nairobi,2020,Mar,12,2020-03,Mar-2020


In [33]:
county_daily_data["date_of_lab_confirmation"] = pd.to_datetime(county_daily_data["date_of_lab_confirmation"])
county_daily_data["Date"] = county_daily_data["date_of_lab_confirmation"]#.dt.date
county_daily_data.set_index("Date", inplace = True)


In [34]:
    data=county_daily_data.loc["2020-03-12" : "2020-06-12"]
    data = data[data["county"] == "Nairobi"].,
    data = data[["county","lab_results", "date_of_lab_confirmation"]].groupby(["county","date_of_lab_confirmation"]).count().reset_index()
    fig = px.line(data, x="date_of_lab_confirmation" , y = "lab_results" , color = "county")
    fig.update_yaxes(title = "Reported Cases", showline=True, linewidth = 0.2, linecolor = "gray", gridcolor = "gainsboro")
    fig.update_xaxes(title = "Date Reported", showgrid=False)
    fig.update_traces(mode = 'lines', marker_color = "#7086CE")
    fig.update_layout(paper_bgcolor="#EDEDED",plot_bgcolor = "#EDEDED",uniformtext_minsize = 3, font_color = "#4F4F4F", bargap =0.05)
 
fig.show()


SyntaxError: invalid syntax (3953024781.py, line 2)

In [ ]:
! ls

Admin2                              covid_monthly_data.csv
Covid-19_cleaned_07-02-2022.csv     data_preprocessing2.ipynb
Covid-19_cleaned_07-02-2022.csv.zip death_cases.csv
Covid-19_cleaned_07-02-2022.txt     kenya
Covid-19_cleaned_07-02-2022.xlsx    kenyan-counties
cases_per_county.csv                new_dash.py
county_cases_spatial.r              plot_county_covid_cases.png
county_daily_data.csv               plot_county_death_cases.png
county_daily_view.py                provice_level_cases.csv
covid_19_kenya.py                   tabs_sidebar_try.py
covid_daily_data.csv                ~$Covid-19_cleaned_07-02-2022.xlsx
covid_data_processed.csv


In [ ]:
filename, extension = os.path.splitext("covid_monthly_data.fastq")
#extension

print(f"{filename}_.fasta")

covid_monthly_data_.fasta
